# Libraries

In [17]:
# from sympy.physics.quantum.qubit import Qubit
# from sympy.physics.quantum.dagger import Dagger
# from sympy.physics.quantum import TensorProduct as sTP
# import sympy.physics.quantum as sq
# import sympy as sp


import qutip as qt

import cvxpy as cp
import numpy as np
import itertools




# # from multiprocessing import Pool

# #Because the arrays are large (over 4GB), they are not getting pickled (something multiprocessing needs to do)
# import pickle4reducer
# import multiprocessing as mp
# ctx = mp.get_context()
# ctx.reducer = pickle4reducer.Pickle4Reducer()



## Partial Trace for cvxpy

In [18]:
##
##https://github.com/cvxgrp/cvxpy/issues/563

import numpy as np
import cvxpy
from cvxpy.expressions.expression import Expression


def expr_as_np_array(cvx_expr):
    if cvx_expr.is_scalar():
        return np.array(cvx_expr)
    elif len(cvx_expr.shape) == 1:
        return np.array([v for v in cvx_expr])
    else:
        # then cvx_expr is a 2d array
        rows = []
        for i in range(cvx_expr.shape[0]):
            row = [cvx_expr[i,j] for j in range(cvx_expr.shape[1])]
            rows.append(row)
        arr = np.array(rows)
        return arr


def np_array_as_expr(np_arr):
    aslist = np_arr.tolist()
    expr = cvxpy.bmat(aslist)
    return expr


def np_partial_trace(rho, dims, axis=0):
    """
    Takes partial trace over the subsystem defined by 'axis'
    rho: a matrix
    dims: a list containing the dimension of each subsystem
    axis: the index of the subsytem to be traced out
    (We assume that each subsystem is square)
    """
    dims_ = np.array(dims)
    # Reshape the matrix into a tensor with the following shape:
    # [dim_0, dim_1, ..., dim_n, dim_0, dim_1, ..., dim_n]
    # Each subsystem gets one index for its row and another one for its column
    reshaped_rho = np.reshape(rho, np.concatenate((dims_, dims_), axis=None))

    # Move the subsystems to be traced towards the end
    reshaped_rho = np.moveaxis(reshaped_rho, axis, -1)
    reshaped_rho = np.moveaxis(reshaped_rho, len(dims)+axis-1, -1)

    # Trace over the very last row and column indices
    traced_out_rho = np.trace(reshaped_rho, axis1=-2, axis2=-1)

    # traced_out_rho is still in the shape of a tensor
    # Reshape back to a matrix
    dims_untraced = np.delete(dims_, axis)
    rho_dim = np.prod(dims_untraced)
    return traced_out_rho.reshape([rho_dim, rho_dim])


def partial_trace(rho, dims, axis=0,debug=False):
    if not isinstance(rho, Expression):
        rho = cvxpy.Constant(shape=rho.shape, value=rho)
    rho_np = expr_as_np_array(rho)
    traced_rho = np_partial_trace(rho_np, dims, axis)
    if(debug):
        print("this is what numpy said")
        print(traced_rho)
        for row in traced_rho:
            for element in row:
                print(element)
    ##########                
    #traced_rho = np_array_as_expr(traced_rho)
    ####################
    # if(debug):
    #     print("this is what the expression became")
    #     print(traced_rho)
    return traced_rho



## Tensor product with cvxpy

In [19]:
def general_kron(a, b):
    """
    Returns a CVXPY Expression representing the Kronecker product of a and b.
    
    At most one of "a" and "b" may be CVXPY Variable objects.
    
    :param a: 2D numpy ndarray, or a CVXPY Variable with a.ndim == 2
    :param b: 2D numpy ndarray, or a CVXPY Variable with b.ndim == 2
    """
    expr = np.kron(a, b)
    num_rows = expr.shape[0]
    rows = [cvxpy.hstack(expr[i,:]) for i in range(num_rows)]
    full_expr = cvxpy.vstack(rows)
    return full_expr

# QuTiP | DI CF with Alice testing rigidity + Weak ET

In [20]:
verbose=True
#fileName = "cache_temp_1"

## Bare Kets

In [21]:
o=qt.basis(2,0)
l=qt.basis(2,1)


I=qt.identity(2)

ι = 0+1j

TP = qt.tensor

def D(obj):
    return obj.dag()

## The unitary and the projector

### Projectors for measurement etc

In [22]:
Π_z0 = o * D(o)
Π_z1 = l * D(l)
display(Π_z0,Π_z1)

def Π_z(i):
    if i==0:
        return Π_z0
    else:
        return Π_z1


x0 = (o + l)/np.sqrt(2)
x1 = (o + l)/np.sqrt(2)


Π_x0 = x0 * D(x0)
Π_x1 = x1 * D(x1)
display(Π_x0,Π_x1)


y0 = (o + ι*l)/np.sqrt(2)
y1 = (o - ι*l)/np.sqrt(2)


Π_y0 = y0 * D(y0)
Π_y1 = y1 * D(y1)
display(Π_y0,Π_y1)


X_hat = o * D(l) + l *D(o)
display(X_hat)

Quantum object: dims = [[2], [2]], shape = (2, 2), type = oper, isherm = True
Qobj data =
[[1. 0.]
 [0. 0.]]

Quantum object: dims = [[2], [2]], shape = (2, 2), type = oper, isherm = True
Qobj data =
[[0. 0.]
 [0. 1.]]

Quantum object: dims = [[2], [2]], shape = (2, 2), type = oper, isherm = True
Qobj data =
[[0.5 0.5]
 [0.5 0.5]]

Quantum object: dims = [[2], [2]], shape = (2, 2), type = oper, isherm = True
Qobj data =
[[0.5 0.5]
 [0.5 0.5]]

Quantum object: dims = [[2], [2]], shape = (2, 2), type = oper, isherm = True
Qobj data =
[[0.5+0.j  0. -0.5j]
 [0. +0.5j 0.5+0.j ]]

Quantum object: dims = [[2], [2]], shape = (2, 2), type = oper, isherm = True
Qobj data =
[[0.5+0.j  0. +0.5j]
 [0. -0.5j 0.5+0.j ]]

Quantum object: dims = [[2], [2]], shape = (2, 2), type = oper, isherm = True
Qobj data =
[[0. 1.]
 [1. 0.]]

### The state, the unitary and the projectors

We define ```U, ρ1, Π``` (qutip objects), ready to use.

In [23]:
#Constructing the unitary

#XRSGYBC
#XRSG


bi = [0,1]


U = None
def f(a,x,r):
    return (a+(x*r))%2


for a,x,r in itertools.product(bi,bi,bi):        
    #print(a,x,r,f(a,x,r))
    if (f(a,x,r)):
        _ = TP(Π_z(a),Π_z(x),Π_z(r),X_hat)        #AXRS
    else:
        _ = TP(Π_z(a),Π_z(x),Π_z(r),I)        #AXRS

    if U is None:
        U = _
        #display(rep(_))
        #display(_)
    else:
        U += _
        #display(_)
    #Qubit(0)
#display(U_II)


ρ = (U*TP(I/2.0,x0*D(x0),x0*D(x0),o*D(o))*D(U)).ptrace([1,2,3])

print(ρ.tr())
display(ρ)

#display(U*D(U))
# print("Constructed the unitary")
# if(verbose): 
#     display(U)


# ρ1=ψ1.ptrace([0,3,4,5,6,7,8,9,10,11])

# Uρ1U=U*ρ1*D(U)

# #        0     12345   678   9
# #ψ1=TP(ψ_ABC,ψ_XYZYZ,ψ_ASS,ψ_R)
# #We also need to trace out S' (the first S above, i.e. index 7)
# trS_Uρ1U= Uρ1U.ptrace([0,1,2,3,4,5,6,8,9])

# if verbose:
#     print("Trace S' of this should equal trace G' σ1")
#     display(Uρ1U)



# ####################Constructing the projector

#XRSGYBC

Π = None
_ = None
#GHZ winning condition
def f(x,y, b,c, r,s,g):
    if( (x+g)%2 == 1 and (s+(x*r)+b+c+1)%2 == x*y*((1+x+y)%2)):
        return 1
    else:
        return 0    

for b,c, x,y, r,s,g in itertools.product(bi,bi,bi, bi,bi,bi, bi):        
    if (f(b,c, x,y, r,s,g)==1):
        #print(a,b,c, x,y,z, g, f(a,b,c,x,y,z,g))
        #XRSGYBC
        _ = TP(Π_z(x),Π_z(r),Π_z(s),Π_z(g),Π_z(y),Π_z(b),Π_z(c))
        if Π is None:
            Π = _
            # print("calculated the symbolic part, working on the representation")
            # Π_ = rep(_)
            # print("completed the representation")
        else:
            Π += _
            # Π_ = rep(_)


display(Π)


# Π = None
# _ = None
# #GHZ winning condition
# def f(a,b,c,x,y,z,g):
#     if( (x+g)%2 == 1 and (a+b+c+1)%2 == (x*y*z)):
#         return 1
#     else:
#         return 0    
# Π_ = None
# #I_ = Π_z0 + Π_z1

# for a,b,c, x,y,z, g in itertools.product(bi,bi,bi, bi,bi,bi, bi):        
#     if (f(a,b,c, x,y,z, g)==1):
#         #print(a,b,c, x,y,z, g, f(a,b,c,x,y,z,g))
#         _ = TP(I,        #A(BC)
#             Π_z(x), Π_z(y),Π_z(z),   #X(YZ) YZ  
#             Π_z(a),I,I,Π_z(g),Π_z(b),Π_z(c)) #A(S)SRGBC

#         if Π is None:
#             Π = _
#             # print("calculated the symbolic part, working on the representation")
#             # Π_ = rep(_)
#             # print("completed the representation")
#         else:
#             Π += _
#             # Π_ = rep(_)


# print("Constructed the projector")            
# if(verbose): 
#     display(Π)
#     # display(Π_)
#     #display(sq.represent(Π))
#     #display(rep(Π))

# #     dill.dump([U,U_I,U_II,Π], open(fileName, "wb"))
# #     print("Saved to disk")


0.9999999999999998


Quantum object: dims = [[2, 2, 2], [2, 2, 2]], shape = (8, 8), type = oper, isherm = True
Qobj data =
[[0.125 0.    0.125 0.    0.125 0.    0.    0.125]
 [0.    0.125 0.    0.125 0.    0.125 0.125 0.   ]
 [0.125 0.    0.125 0.    0.125 0.    0.    0.125]
 [0.    0.125 0.    0.125 0.    0.125 0.125 0.   ]
 [0.125 0.    0.125 0.    0.125 0.    0.    0.125]
 [0.    0.125 0.    0.125 0.    0.125 0.125 0.   ]
 [0.    0.125 0.    0.125 0.    0.125 0.125 0.   ]
 [0.125 0.    0.125 0.    0.125 0.    0.    0.125]]

Quantum object: dims = [[2, 2, 2, 2, 2, 2, 2], [2, 2, 2, 2, 2, 2, 2]], shape = (128, 128), type = oper, isherm = True
Qobj data =
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 1. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]

# The SDP itself

## Provisional implementation (doesn't quite work)

In [24]:
#should ideally be hermitian

#XRSGYBC
ρ2 = cp.Variable((2**7,2**7), hermitian=True)

#XRSG
ρ1 = cp.Variable((2**4,2**4),hermitian=True)


#constraints += [cp.trace(σ1)==1,cp.trace(σ2)==1]


#The non-trivial constraints

#So what I've done here is the following
#When converting back from numpy, it was using Vstac and Hstac
#This seemed to make everything inefficient
#So now, I just return the numpy expression for partial_trace
#and then to trace again, I use the numpy expression

trG_ρ1 = partial_trace(ρ1,[2]*4,3) #XRS(G)

trC_ρ2 = partial_trace(ρ2,[2]*7,6) #XRSGYB(C)
trBC_ρ2 = np_partial_trace(trC_ρ2,[2]*6,5) #XRSGY(B)


# trB_σ2=partial_trace(σ2,[2]*10,8) #trace out C'; AXYZASR(B)CG
# # trBC_σ2=np_array_as_expr(np_partial_trace(trB_σ2,[2]*9,8)) #trace out C'; AXYZASR(C)G
# trBC_σ2=np_partial_trace(trB_σ2,[2]*9,8) #trace out C'; AXYZASR(C)G

# trG_σ1=partial_trace(σ1,[2]*10,9) #trace out G; AXYZYZASR(G)

# trY_σ1=partial_trace(σ1,[2]*10,2) #trace out Y; AX(Y)ZYZASRG
# # trYZ_σ1=np_array_as_expr(np_partial_trace(trY_σ1,[2]*9,2)) #trace out Z; AX(Z)YZASRG
# trYZ_σ1=np_partial_trace(trY_σ1,[2]*9,2) #trace out Z; AX(Z)YZASRG

In [83]:
print(trBC_ρ2[0,0])

var223606[0, 0] + var223606[1, 1] + var223606[2, 2] + var223606[3, 3]


In [25]:
#constraints = [σ1>>0] #[σ1>>0, σ2>>0]
constraints = [ρ1>>0,ρ2>>0]

#Just to test, I am using the numpy expression
#And adding the condition for each element
#cvxpy wasn't accepting these numpy matrices directly

#print(trYZ_σ1[0,0])

ρ1_Y = general_kron(ρ1,np.eye(2)/2.0)



print("Working")

#BRSGY BC
n=len(trBC_ρ2) #int(len(trBC_ρ2)/(2**0))

for i in range(n):
    print(i,end='\r')
    for j in range(int(n)):
        constraints += [trBC_ρ2[i,j] == ρ1_Y[i,j]] #np.eye(n)[i,j]] #ρ1_Y[i,j]] #np.kron(ρ1,np.eye(2)/2.0)[i,j]]
print(n)
print("completed 1 of 2")

#XRS G
n=len(trG_ρ1) # int(len(trG_ρ1)/(2**0))
print("Working")
for i in range(n):
    print(i,end='\r')
    for j in range(n): #range(int(len(trG_σ1)/2)):
        constraints += [trG_ρ1[i,j] == ρ[i,j]]
print(n)
print("completed 2 of 2")




print(constraints[4])

# constraints += [trG_σ1 == trS_Uρ1U.data, trBC_σ2 == trYZ_σ1 ]

#constraints += [ cp.trace(σ1) == cp.trace(trS_Uρ1U),cp.trace(σ2)==cp.trace(σ1) ]


Working
32
completed 1 of 2
Working
8
completed 2 of 2
var37696[0, 8] + var37696[1, 9] + var37696[2, 10] + var37696[3, 11] == Vstack(Hstack(var37697 @ Promote(0.5, (16, 16)), var37697 @ Promote(0.0, (16, 16))), Hstack(var37697 @ Promote(0.0, (16, 16)), var37697 @ Promote(0.5, (16, 16))))[0, 2]


In [ ]:
#trG_σ1
#trS_Uρ1U.data
#print(trBC_σ2[0,0])

In [ ]:
#trYZ_σ1[1,1]

In [27]:
prob = cp.Problem(cp.Maximize(cp.trace(cp.real(Π.data @ ρ2 ))),constraints)
#prob = cp.Problem(cp.Maximize(cp.trace(ρ2)),constraints)
prob.solve(solver=cp.MOSEK,verbose=True) #MOSEK or SCS




Problem
  Name                   :                 
  Objective sense        : min             
  Type                   : CONIC (conic optimization problem)
  Constraints            : 68736           
  Cones                  : 0               
  Scalar variables       : 25032           
  Matrix variables       : 2               
  Integer variables      : 0               

Optimizer started.
Problem
  Name                   :                 
  Objective sense        : min             
  Type                   : CONIC (conic optimization problem)
  Constraints            : 68736           
  Cones                  : 0               
  Scalar variables       : 25032           
  Matrix variables       : 2               
  Integer variables      : 0               

Optimizer  - threads                : 20              
Optimizer  - solved problem         : the primal      
Optimizer  - Constraints            : 66560
Optimizer  - Cones                  : 1
Optimizer  - Scalar variabl

0.437581495612759

In [ ]:
MOSEK solved the same problem 

Problem
  Name                   :                 
  Objective sense        : min             
  Type                   : CONIC (conic optimization problem)
  Constraints            : 68736           
  Cones                  : 0               
  Scalar variables       : 25032           
  Matrix variables       : 2               
  Integer variables      : 0               

Optimizer started.
Problem
  Name                   :                 
  Objective sense        : min             
  Type                   : CONIC (conic optimization problem)
  Constraints            : 68736           
  Cones                  : 0               
  Scalar variables       : 25032           
  Matrix variables       : 2               
  Integer variables      : 0               

Optimizer  - threads                : 20              
Optimizer  - solved problem         : the primal      
Optimizer  - Constraints            : 66560
Optimizer  - Cones                  : 1
Optimizer  - Scalar variables       : 15553             conic                  : 15553           
Optimizer  - Semi-definite variables: 2                 scalarized             : 33424           
Factor     - setup time             : 442.98            dense det. time        : 0.00            
Factor     - ML order time          : 103.97            GP order time          : 0.00            
Factor     - nonzeros before factor : 2.15e+09          after factor           : 2.15e+09        
Factor     - dense dim.             : 2                 flops                  : 9.38e+13        
ITE PFEAS    DFEAS    GFEAS    PRSTATUS   POBJ              DOBJ              MU       TIME  
0   1.0e+00  1.0e+00  1.0e+00  0.00e+00   -2.500000000e-01  -2.500000000e-01  1.0e+00  443.43
1   9.1e-02  9.1e-02  3.1e-02  1.42e-01   -4.185348954e+00  -4.188761963e+00  9.1e-02  1016.71
2   6.5e-03  6.5e-03  5.5e-04  2.42e+00   -7.507011471e-01  -7.475611122e-01  6.5e-03  1578.62
3   1.1e-03  1.1e-03  5.6e-05  7.84e-02   -4.867122639e-01  -4.853440212e-01  1.1e-03  2151.49
4   9.7e-05  9.7e-05  1.7e-06  6.81e-01   -4.469824925e-01  -4.467984298e-01  9.7e-05  2728.09
5   2.4e-05  2.4e-05  2.2e-07  3.16e-01   -4.423597736e-01  -4.423178199e-01  2.4e-05  3277.42
6   4.4e-06  4.4e-06  2.4e-08  6.02e-01   -4.399315634e-01  -4.399113787e-01  4.4e-06  3838.16
7   1.3e-06  1.3e-06  5.4e-09  4.32e-01   -4.389888039e-01  -4.389762957e-01  1.3e-06  4385.85
8   3.5e-07  3.5e-07  1.1e-09  4.19e-01   -4.384073633e-01  -4.383997570e-01  3.5e-07  4932.59
9   8.9e-08  8.7e-08  1.9e-10  4.69e-01   -4.380706226e-01  -4.380662046e-01  8.7e-08  5475.14
10  2.6e-08  1.4e-08  2.4e-11  3.03e-01   -4.378150050e-01  -4.378121307e-01  1.4e-08  6032.71
11  1.7e-08  9.0e-09  1.5e-11  3.72e-01   -4.377679832e-01  -4.377654031e-01  9.1e-09  6568.00
12  1.1e-08  1.6e-09  1.9e-12  4.66e-01   -4.376601065e-01  -4.376588362e-01  1.6e-09  7132.28
13  7.3e-09  1.0e-09  1.1e-12  1.40e-01   -4.376333733e-01  -4.376321051e-01  1.0e-09  9231.19
14  2.4e-09  3.4e-10  2.2e-13  3.58e-01   -4.375814956e-01  -4.375807981e-01  2.6e-10  9789.46
15  2.4e-09  3.4e-10  2.2e-13  4.58e-01   -4.375814956e-01  -4.375807981e-01  2.6e-10  10371.33
16  2.4e-09  3.4e-10  2.2e-13  9.92e-01   -4.375814956e-01  -4.375807981e-01  2.6e-10  10932.64
Optimizer terminated. Time: 11497.03


Interior-point solution summary
  Problem status  : PRIMAL_AND_DUAL_FEASIBLE
  Solution status : OPTIMAL
  Primal.  obj: -4.3758149561e-01   nrm: 1e-01    Viol.  con: 1e-07    var: 0e+00    barvar: 0e+00  
  Dual.    obj: -4.3758079809e-01   nrm: 5e+02    Viol.  con: 0e+00    var: 1e-08    barvar: 2e-08  
0.437581495612759

In [ ]:
Post Jamie Magic #1

----------------------------------------------------------------------------
	SCS v2.1.2 - Splitting Conic Solver
	(c) Brendan O'Donoghue, Stanford University, 2012
----------------------------------------------------------------------------
Lin-sys: sparse-direct, nnz in A = 63632
eps = 1.00e-04, alpha = 1.50, max_iters = 5000, normalize = 1, scale = 1.00
acceleration_lookback = 10, rho_x = 1.00e-03
Variables n = 25032, constraints m = 35600
Cones:	primal zero / dual free vars: 2176
	sd vars: 33424, sd blks: 2
Setup time: 1.07e-01s
----------------------------------------------------------------------------
 Iter | pri res | dua res | rel gap | pri obj | dua obj | kap/tau | time (s)
----------------------------------------------------------------------------
     0| 4.62e+20  2.03e+21  1.00e+00 -2.34e+21  5.02e+21  1.47e+21  4.72e-02 
   100| 1.68e-03  7.53e-03  4.59e-04 -4.36e-01 -4.35e-01  6.35e-17  1.95e+00 
   200| 2.22e-03  7.93e-03  7.24e-04 -4.40e-01 -4.42e-01  1.56e-16  3.92e+00 
   300| 4.12e-04  1.46e-03  5.52e-04 -4.38e-01 -4.37e-01  2.20e-18  5.91e+00 
   400| 3.97e+17  1.28e+18  1.55e-03  2.71e+17  2.70e+17  1.68e+18  7.87e+00 
   500| 1.00e-04  3.98e-04  2.32e-04 -4.38e-01 -4.38e-01  4.24e-16  9.78e+00 
   600| 9.82e-04  3.21e-03  2.12e-03 -4.37e-01 -4.41e-01  2.50e-16  1.17e+01 
   700| 6.53e-03  2.07e-02  1.47e-02 -4.34e-01 -4.62e-01  1.53e-16  1.35e+01 
   800| 3.59e-04  1.15e-03  7.86e-04 -4.37e-01 -4.39e-01  3.79e-16  1.54e+01 
   900| 3.98e-04  1.39e-03  6.97e-04 -4.37e-01 -4.39e-01  2.41e-16  1.73e+01 
  1000| 2.23e-03  7.10e-03  4.83e-03 -4.36e-01 -4.45e-01  2.59e-16  1.92e+01 
  1100| 1.67e+15  5.39e+15  3.88e-03  1.14e+15  1.13e+15  7.03e+15  2.11e+01 
  1200| 2.99e+15  9.63e+15  1.99e-03  2.03e+15  2.02e+15  1.25e+16  2.30e+01 
  1300| 5.01e-02  1.71e-01  8.12e-04 -4.41e-01 -4.43e-01  9.25e-17  2.50e+01 
  1400| 8.06e-04  2.62e-03  1.76e-03 -4.37e-01 -4.40e-01  3.34e-18  2.69e+01 
  1500| 1.95e-03  6.39e-03  4.55e-03 -4.37e-01 -4.46e-01  1.31e-16  2.88e+01 
  1600| 1.43e+15  4.60e+15  1.82e-03  9.44e+14  9.41e+14  5.87e+15  3.07e+01 
  1700| 2.97e+15  9.55e+15  1.81e-03  2.02e+15  2.01e+15  1.25e+16  3.26e+01 
  1800| 3.51e-04  1.15e-03  7.18e-04 -4.38e-01 -4.39e-01  1.66e-16  3.46e+01 
  1900| 7.83e-04  2.55e-03  1.67e-03 -4.37e-01 -4.40e-01  2.19e-16  3.65e+01 
  2000| 1.75e-03  5.44e-03  4.20e-03 -4.37e-01 -4.45e-01  4.12e-16  3.84e+01 
  2100| 2.21e-03  7.12e-03  4.93e-03 -4.36e-01 -4.45e-01  3.74e-17  4.03e+01 
  2200| 9.27e-04  3.00e-03  2.06e-03 -4.37e-01 -4.41e-01  3.82e-16  4.22e+01 
  2300| 8.87e-04  2.66e-03  2.33e-03 -4.37e-01 -4.41e-01  3.95e-16  4.41e+01 
  2400| 4.75e-04  1.50e-03  8.38e-04 -4.37e-01 -4.38e-01  6.16e-17  4.61e+01 
  2500| 6.44e-04  2.15e-03  1.30e-03 -4.37e-01 -4.40e-01  1.62e-16  4.80e+01 
  2600| 3.90e-02  1.39e-01  8.03e-04 -4.39e-01 -4.38e-01  2.65e-16  4.99e+01 
  2700| 6.06e-04  1.93e-03  1.39e-03 -4.37e-01 -4.40e-01  1.98e-16  5.18e+01 
  2800| 8.83e-04  2.89e-03  1.85e-03 -4.37e-01 -4.40e-01  6.59e-17  5.37e+01 
  2900| 5.84e-05  2.52e-04  1.36e-05 -4.37e-01 -4.37e-01  1.27e-16  5.57e+01 
  3000| 4.24e+15  1.37e+16  8.99e-04  2.81e+15  2.82e+15  1.75e+16  5.76e+01 
  3100| 7.00e-05  2.10e-04  5.89e-05 -4.37e-01 -4.38e-01  4.36e-16  5.96e+01 
  3160| 2.50e-05  6.26e-05  4.69e-06 -4.38e-01 -4.38e-01  6.04e-17  6.08e+01 
----------------------------------------------------------------------------
Status: Solved
Timing: Solve time: 6.08e+01s
	Lin-sys: nnz in L factor: 133624, avg solve time: 6.50e-04s
	Cones: avg projection time: 1.36e-02s
	Acceleration: avg step time: 3.16e-03s
----------------------------------------------------------------------------
Error metrics:
dist(s, K) = 7.5980e-09, dist(y, K*) = 3.1513e-09, s'y/|s||y| = -2.0472e-10
primal res: |Ax + s - b|_2 / (1 + |b|_2) = 2.4997e-05
dual res:   |A'y + c|_2 / (1 + |c|_2) = 6.2568e-05
rel gap:    |c'x + b'y| / (1 + |c'x| + |b'y|) = 4.6943e-06
----------------------------------------------------------------------------
c'x = -0.4375, -b'y = -0.4375
============================================================================
0.4375148594226565

# Temporary

In [ ]:
#Uttam Bhai's formula
#उत्तम भाई

for i in [True,False]:
    for j in [True,False]:
        for k in [True,False]:
            print ((i or (j and k)), ((i or j) and (i or k)))            